#### CASO TARJETA FIDELIZACIÓN

Teniendo un dataset con las siguientes columnas:
- CONTRATO TARJETA	Código del contrato de la tarjeta de crédito
- CCLIEN TITULAR TARJETA	Código cliente
- ESTADO CLIENTE	Estado cliente: A Activo, C Cancelado, I Inactivo
- COD_EDAD	Edad del cliente
- SALDO CUENTA	Saldo de la cuenta donde está domiciliada la tarjeta
- IMPORTE CRÉDITO CONSUMO	Capital pendiente del crédito al consumo del cliente a final de mes
- GASTO COMERCIOS TARJETA	Gasto en comercios del mes
- MOVIMIENTOS COMERCIO TARJETA	Número de movimientos realizados en el mes en comercio
- GASTO CAJEROS TARJETA	Gasto en cajeros del mes
- MOVIMIENTOS CAJERO TARJETA	Número de movimientos realizados en el mes en cajero

Disponemos de una tarjeta de fidelización que te da puntos por compras, en función de ciertas condiciones:											
5 puntos por movimiento en comercio siempre que tu gasto total con la tarjeta sea superior a 1.200€											
Queremos responder varias preguntas:											
1) ¿Cuál es la tenencia de préstamos al consumo / saldo medio?											
2) ¿Cuál es el saldo medio de préstamos al consumo en clientes de 30 a 59 años?											
3) ¿Cuál es gasto medio en comercios con tarjeta?											
4) ¿Cuántos clientes tienen gasto en cajeros?											
5) ¿Qué porcentaje de clientes no consiguen puntos? ¿Cuántos puntos en total consiguen los que sí que los logran?											

### Abrir los fichero de datos

In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformación Digital/6. CASO TARJETA FIDELIZACIÓN.xlsx - DATOS BASE.csv')
df

,CONTRATO TARJETA,CCLIEN TITULAR TARJETA,ESTADO CLIENTE,COD_EDAD,SALDO CUENTA,IMPORTE CRÉDITO CONSUMO,GASTO COMERCIOS TARJETA,MOVIMIENTOS COMERCIO TARJETA,GASTO CAJEROS TARJETA,MOVIMIENTOS CAJERO TARJETA
0,701276896,829829,A,35,55649,0,1962,52,1610,16
1,2339857631,569268,A,43,3133,3042,1677,19,790,7
2,3166250352,667899,A,71,1248,0,107,5,545,27
3,5393713901,580616,A,21,71615,0,2213,39,1980,19
4,4783796252,572291,A,70,4007,5512,1065,18,2357,23
...,...,...,...,...,...,...,...,...,...,...
995,7818424772,282264,A,21,89839,0,1538,18,1307,13
996,6666485162,537047,A,30,7614,0,635,5,762,7
997,6532353450,800404,A,57,4992,6283,1967,54,183,1
998,8534512084,516649,A,84,8377,0,2042,38,259,2


### Filtros

Filtramos aquellos clientes que hayan cancelado

In [16]:
print('Número de registros inicial: ', df.shape[0])
df_activos = df[(df['ESTADO CLIENTE'] != 'C')]
print('Número de registros filtrando por no cancelados: ', df_activos.shape[0])

Número de registros inicial:  1000
Número de registros filtrando por no cancelados:  946


### Limpieza de datos

Comprobamos si hay missing values y los sustituimos

In [17]:
df_activos.isna().sum()

CONTRATO TARJETA                0
CCLIEN TITULAR TARJETA          0
ESTADO CLIENTE                  0
COD_EDAD                        0
SALDO CUENTA                    0
IMPORTE CRÉDITO CONSUMO         0
GASTO COMERCIOS TARJETA         0
MOVIMIENTOS COMERCIO TARJETA    0
GASTO CAJEROS TARJETA           0
MOVIMIENTOS CAJERO TARJETA      0
dtype: int64

### Variables contador
Generamos variables de consumo y gasto en comercios y cajeros:

In [18]:
import numpy as np

df_activos['CONSUMO'] = np.where(df_activos['IMPORTE CRÉDITO CONSUMO'] > 0, 1, 0)
df_activos['COMERCIOS'] = np.where(df_activos['GASTO COMERCIOS TARJETA'] > 0, 1, 0)
df_activos['CAJEROS'] = np.where(df_activos['GASTO CAJEROS TARJETA'] > 0, 1, 0)

<ipython-input-18-b6d86b96025a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activos['CONSUMO'] = np.where(df_activos['IMPORTE CRÉDITO CONSUMO'] > 0, 1, 0)
<ipython-input-18-b6d86b96025a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activos['COMERCIOS'] = np.where(df_activos['GASTO COMERCIOS TARJETA'] > 0, 1, 0)
<ipython-input-18-b6d86b96025a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [19]:
df_activos.agg({'CONSUMO': 'sum',
            'IMPORTE CRÉDITO CONSUMO': 'sum',
            'COMERCIOS': 'sum',
            'GASTO COMERCIOS TARJETA': 'sum',
            'CAJEROS': 'sum',
            'GASTO CAJEROS TARJETA': 'sum'})

CONSUMO                        419
IMPORTE CRÉDITO CONSUMO    1964154
COMERCIOS                      781
GASTO COMERCIOS TARJETA     873085
CAJEROS                        908
GASTO CAJEROS TARJETA      1175486
dtype: int64

1) ¿Cuál es la tenencia de préstamos al consumo / saldo medio?

In [20]:
df_activos.agg({'CONSUMO': lambda x: x.sum() / df_activos.shape[0] * 100,
                'IMPORTE CRÉDITO CONSUMO': lambda x: x.sum() / df_activos[(df_activos['CONSUMO'] == 1)].shape[0]})

CONSUMO                      44.291755
IMPORTE CRÉDITO CONSUMO    4687.718377
dtype: float64

2) ¿Cuál es el saldo medio de préstamos al consumo en clientes de 30 a 59 años?

In [21]:
df_activos_edad = df_activos[(df_activos['COD_EDAD'] >= 30) & (df_activos['COD_EDAD'] <= 59)]

df_activos_edad.agg({'CONSUMO': ['sum', lambda x: x.sum() / df_activos_edad.shape[0] * 100],
                'IMPORTE CRÉDITO CONSUMO': ['sum', lambda x: x.sum() / df_activos_edad[(df_activos_edad['CONSUMO'] == 1)].shape[0]]})

,CONSUMO,IMPORTE CRÉDITO CONSUMO
sum,177.000000,828436.000000
<lambda>,43.703704,4680.429379


3) ¿Cuál es gasto medio en comercios con tarjeta?

In [25]:
df_activos.agg({'COMERCIOS': 'sum',
                'GASTO COMERCIOS TARJETA': ['sum', lambda x: x.sum() / df_activos[(df_activos['COMERCIOS'] == 1)].shape[0]]})

,COMERCIOS,GASTO COMERCIOS TARJETA
sum,781.0,873085.00000
<lambda>,NaN,1117.90653


4) ¿Cuántos clientes tienen gasto en cajeros?

In [27]:
df_activos.agg({'CAJEROS': ['sum', lambda x: x.sum() / df_activos.shape[0] * 100],
                'GASTO CAJEROS TARJETA': 'sum'})

,CAJEROS,GASTO CAJEROS TARJETA
sum,908.000000,1175486.0
<lambda>,95.983087,NaN


5) ¿Qué porcentaje de clientes no consiguen puntos? ¿Cuántos puntos en total consiguen los que sí que los logran?

* Condición para conseguir puntos:											
5 puntos por movimiento en comercio siempre que tu gasto total con la tarjeta sea superior a 1.200€											
											
Clientes que cumplen el filtro y tienen gasto total con la tarjeta superior a 1.200€											
Creamos una variable contador de esta condición											
Creamos una variable que sume los puntos en función de la condición											

In [29]:
df_activos['OBTIENE_PUNTOS'] = np.where((df_activos['GASTO CAJEROS TARJETA'] + df_activos['GASTO COMERCIOS TARJETA']) > 1200, 1, 0)
df_activos['OBTIENE_PUNTOS']

<ipython-input-29-16e05b667cd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activos['OBTIENE_PUNTOS'] = np.where((df_activos['GASTO CAJEROS TARJETA'] + df_activos['GASTO COMERCIOS TARJETA']) > 1200, 1, 0)


0      1
1      1
2      0
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Name: OBTIENE_PUNTOS, Length: 946, dtype: int64

In [40]:
df_puntos = df_activos[(df_activos['OBTIENE_PUNTOS'] > 0)]
porcentaje_cuentas_sin_puntos = (df_activos.shape[0] - df_puntos.shape[0])/ df_activos.shape[0] * 100
print('Cuentas que consiguen puntos: ', df_puntos.shape[0])
print("Puntos obtenidos: ", (df_puntos['MOVIMIENTOS COMERCIO TARJETA'] * 5).sum())
print(f'Cuentas sin puntos: {porcentaje_cuentas_sin_puntos}%')

Cuentas que consiguen puntos:  747
Puntos obtenidos:  74480
Cuentas sin puntos: 21.035940803382662%
